In [146]:
import polars as pl
import tarfile
import os
import datetime as dt
from pathlib import Path
from tqdm import tqdm

from src.preprocessing import preprocess_all_tickers, preprocess_ticker, convert_xltime_to_timestamp, fill_missing_minutes

# Autorealod extension for Jupyter Notebooks
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_amzn = pl.read_parquet("data/AMZN.OQ/2016-01-04-AMZN.OQ-bbo.parquet")
df_amzn

xltime,bid-price,bid-volume,ask-price,ask-volume
f64,f64,i32,f64,i32
42373.375006,0.0,0,669.8,1
42373.375014,286.0,1,669.8,1
42373.375616,286.0,1,0.0,0
42373.380269,286.0,1,695.0,1
42373.380503,286.0,1,0.0,0
…,…,…,…,…
42373.953085,637.0,1,638.75,1
42373.96403,637.0,1,638.1,1
42373.96403,637.0,1,638.75,1


In [17]:
preprocess_all_tickers()

Preprocessing tickers:   0%|          | 0/9 [00:00<?, ?it/s]

Error preprocessing XOM.OQ: No parquet files found for XOM.OQ
Error preprocessing BRK.B.OQ: No parquet files found for BRK.B.OQ
Error preprocessing FOX.N: No parquet files found for FOX.N
Error preprocessing KHC.N: No parquet files found for KHC.N
Error preprocessing TXN.N: No parquet files found for TXN.N
Error preprocessing ABBV.OQ: No parquet files found for ABBV.OQ
Error preprocessing WBA.N: No parquet files found for WBA.N
Error preprocessing FB.N: No parquet files found for FB.N
Error preprocessing ORCL.OQ: No parquet files found for ORCL.OQ
93 tickers preprocessed successfully over 102 attempted.


In [8]:
df_1 = pl.read_parquet("data/preprocessed/SP100/bbo/AGN.N.parquet")
df_1

timestamp,mid_price,mid_price_return
"datetime[μs, America/New_York]",f64,f64
2015-01-02 09:33:00 EST,213.905,0.001498
2015-01-02 09:34:00 EST,213.97,0.000304
2015-01-02 09:35:00 EST,214.33,0.001682
2015-01-02 09:36:00 EST,214.28,-0.000233
2015-01-02 09:37:00 EST,214.335,0.000257
…,…,…
2017-03-31 15:55:00 EDT,239.62,0.000397
2017-03-31 15:56:00 EDT,239.445,-0.00073
2017-03-31 15:57:00 EDT,239.46,0.000063


In [18]:
df_2 = pl.read_parquet("data/preprocessed/SP100/bbo/AAPL.OQ.parquet")
df_2

timestamp,mid_price,mid_price_return
"datetime[μs, America/New_York]",f64,f64
2015-01-02 09:31:00 EST,111.295,0.000404
2015-01-02 09:32:00 EST,111.29,-0.000045
2015-01-02 09:33:00 EST,111.175,-0.001033
2015-01-02 09:34:00 EST,111.05,-0.001124
2015-01-02 09:35:00 EST,111.22,0.001531
…,…,…
2017-04-12 15:55:00 EDT,141.685,-0.000141
2017-04-12 15:56:00 EDT,141.725,0.000282
2017-04-12 15:57:00 EDT,141.695,-0.000212


In [ ]:
import polars as pl
from pathlib import Path

folder = Path("data/preprocessed/SP100/bbo")
files = list(folder.glob("*.parquet"))
problematic_files = 0

# Expected number of rows per day
# From 9:31 to 15:59 inclus, il y a (15-9)*60 + (59-31+1) = 389 minutes
EXPECTED_ROWS_PER_DAY = 389

for f in files:
    df = pl.read_parquet(f, columns=["timestamp"])
    # Ensure timestamp is in datetime format
    df = df.with_columns(pl.col("timestamp").dt.date().alias("date"))

    # Count rows per day
    rows_per_day = df.group_by("date").len().rename({"len": "num_rows"})
    overmatched_days = rows_per_day.filter(pl.col("num_rows") > EXPECTED_ROWS_PER_DAY)
    undermatched_days = rows_per_day.filter(pl.col("num_rows") < EXPECTED_ROWS_PER_DAY)

    if overmatched_days.height > 0 or undermatched_days.height > 0:
        print(f"File {f.name} has {overmatched_days.height} days with extra rows and {undermatched_days.height} days with missing rows")
        problematic_files += 1
    else:
        print(f"File {f.name} is fine (all days have {EXPECTED_ROWS_PER_DAY} rows).")

# print num of problematic files
print(f"Total problematic files: {problematic_files} out of {len(files)}")

File EXC.N.parquet has 2 days with extra rows and 143 days with missing rows
File MA.N.parquet has 2 days with extra rows and 172 days with missing rows
File GOOG.OQ.parquet has 3 days with extra rows and 93 days with missing rows
File GOOGL.OQ.parquet has 1 days with extra rows and 72 days with missing rows
File PEP.N.parquet has 0 days with extra rows and 185 days with missing rows
File AMGN.OQ.parquet has 10 days with extra rows and 12 days with missing rows
File BK.N.parquet has 2 days with extra rows and 197 days with missing rows
File F.N.parquet has 2 days with extra rows and 223 days with missing rows
File C.N.parquet has 3 days with extra rows and 228 days with missing rows
File CVS.N.parquet has 1 days with extra rows and 171 days with missing rows
File VZ.N.parquet has 5 days with extra rows and 152 days with missing rows
File DIS.N.parquet has 2 days with extra rows and 219 days with missing rows
File JNJ.N.parquet has 0 days with extra rows and 172 days with missing rows
F

In [138]:
undermatched_days

date,num_rows
date,u32


In [163]:
preprocess_ticker("EXC.N")

2025-11-25 16:16:18,530 | INFO | === Preprocessing ticker EXC.N ===
2025-11-25 16:16:18,560 | INFO | [EXC.N] Extracting archives...
2025-11-25 16:16:18,688 | DEBUG | [EXC.N] Found 3 archives.
2025-11-25 16:16:22,632 | DEBUG | [EXC.N] Extracted EXC.N_2015_bbo.tar
2025-11-25 16:16:26,181 | DEBUG | [EXC.N] Extracted EXC.N_2016_bbo.tar
2025-11-25 16:16:26,730 | DEBUG | [EXC.N] Extracted EXC.N_2017_bbo.tar
2025-11-25 16:16:26,734 | INFO | Loading parquet files in data/extracted/SP100/bbo/EXC.N...
2025-11-25 16:16:26,857 | DEBUG | Loaded 2015-01-02-EXC.N-bbo.parquet (204098 rows)
2025-11-25 16:16:27,026 | DEBUG | Loaded 2015-01-05-EXC.N-bbo.parquet (188382 rows)
2025-11-25 16:16:27,166 | DEBUG | Loaded 2015-01-06-EXC.N-bbo.parquet (247491 rows)
2025-11-25 16:16:27,211 | DEBUG | Loaded 2015-01-07-EXC.N-bbo.parquet (160463 rows)
2025-11-25 16:16:27,273 | DEBUG | Loaded 2015-01-08-EXC.N-bbo.parquet (131667 rows)
2025-11-25 16:16:27,372 | DEBUG | Loaded 2015-01-09-EXC.N-bbo.parquet (175033 rows)

timestamp,mid_price_return
"datetime[μs, America/New_York]",f64
2015-01-02 09:32:00 EST,0.0
2015-01-02 09:33:00 EST,0.0
2015-01-02 09:34:00 EST,0.0
2015-01-02 09:35:00 EST,0.000268
2015-01-02 09:36:00 EST,0.00362
…,…
2017-03-31 15:56:00 EDT,-0.000277
2017-03-31 15:57:00 EDT,-0.000833
2017-03-31 15:58:00 EDT,-0.000278


In [164]:
df_ticker = pl.read_parquet("data/preprocessed/SP100/bbo/EXC.N.parquet")
df_ticker

timestamp,mid_price_return
"datetime[μs, America/New_York]",f64
2015-01-02 09:32:00 EST,0.0
2015-01-02 09:33:00 EST,0.0
2015-01-02 09:34:00 EST,0.0
2015-01-02 09:35:00 EST,0.000268
2015-01-02 09:36:00 EST,0.00362
…,…
2017-03-31 15:56:00 EDT,-0.000277
2017-03-31 15:57:00 EDT,-0.000833
2017-03-31 15:58:00 EDT,-0.000278


In [165]:
unique_dates = df_ticker.select(pl.col("timestamp").dt.date().unique())
print(unique_dates.height * 389)

219785


In [ ]:
import pandas_market_calendars as mcal
import pandas as pd

# Crée un calendrier pour le NYSE
nyse = mcal.get_calendar('NYSE')

# Dates de début et fin
start_date = pd.Timestamp('2015-01-01')
end_date = pd.Timestamp('2017-12-31')

# Obtenir les jours de trading
schedule = nyse.schedule(start_date=start_date, end_date=end_date)
trading_days = schedule.index.to_list()

print(trading_days)  # liste de Timestamp


[Timestamp('2015-01-02 00:00:00'), Timestamp('2015-01-05 00:00:00'), Timestamp('2015-01-06 00:00:00'), Timestamp('2015-01-07 00:00:00'), Timestamp('2015-01-08 00:00:00'), Timestamp('2015-01-09 00:00:00'), Timestamp('2015-01-12 00:00:00'), Timestamp('2015-01-13 00:00:00'), Timestamp('2015-01-14 00:00:00'), Timestamp('2015-01-15 00:00:00'), Timestamp('2015-01-16 00:00:00'), Timestamp('2015-01-20 00:00:00'), Timestamp('2015-01-21 00:00:00'), Timestamp('2015-01-22 00:00:00'), Timestamp('2015-01-23 00:00:00'), Timestamp('2015-01-26 00:00:00'), Timestamp('2015-01-27 00:00:00'), Timestamp('2015-01-28 00:00:00'), Timestamp('2015-01-29 00:00:00'), Timestamp('2015-01-30 00:00:00'), Timestamp('2015-02-02 00:00:00'), Timestamp('2015-02-03 00:00:00'), Timestamp('2015-02-04 00:00:00'), Timestamp('2015-02-05 00:00:00'), Timestamp('2015-02-06 00:00:00'), Timestamp('2015-02-09 00:00:00'), Timestamp('2015-02-10 00:00:00'), Timestamp('2015-02-11 00:00:00'), Timestamp('2015-02-12 00:00:00'), Timestamp('20